# This is a quick notebook to play with caching decorators provided in `functools`

Inspired by https://www.youtube.com/watch?v=EsUTO4Xzehg

I knew that function invocations in python were slow but I didn't realize how slow they actually were before seeing how long the simple recursive fibonacci series took without caching.





In [11]:
import time
import functools

## 1. without cache

In [2]:
def fib(n):
    if n==0 or n==1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [3]:
t1 = time.time()
print([fib(f) for f in range(35)])
t2 = time.time()

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465]


In [4]:
print(t2-t1)

22.815624237060547


More than 20 seconds !

## 2. with `functools.lru_cache`

In [5]:
import functools

In [6]:
@functools.lru_cache(maxsize=5)
def cached_fib(n):
    if n==0 or n==1:
        return 1
    else:
        return cached_fib(n-1) + cached_fib(n-2)

In [7]:
t1 = time.time()
print([cached_fib(f) for f in range(35)])
t2 = time.time()

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465]


In [8]:
print(t2-t1)

0.0012989044189453125


In [9]:
cached_fib.cache_info()

CacheInfo(hits=66, misses=35, maxsize=5, currsize=5)

# Other caching packages:


* [`cachetools`](https://cachetools.readthedocs.io/en/stable/#) has more strategies for replacing items if cache is full (least frequently used, random replacement, time-to-live)
* also look at redis as a cache

## Can we apply this to images ?
The cache needs to create keys from the arguments. How does this work if we pass in large numpy arrays? Will they get hashed ?

In [1]:
import numpy as np

In [5]:
volume = np.random.randn(100,100,100)

In [7]:
def my_mean(vol):
    return [np.mean(vol, axis=i) for i in range(vol.ndim)]

In [10]:
%%timeit 
[my_mean(volume) for i in range(10)]


54.3 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
@functools.lru_cache(maxsize=5)
def cached_my_mean(vol):
    return [np.mean(vol, axis=i) for i in range(vol.ndim)]

In [15]:
%%timeit 
[cached_my_mean(volume) for i in range(10)]

TypeError: unhashable type: 'numpy.ndarray'

## :-( doesn't work because `numpy.ndarray` is an unhashable type)